In [1]:
from monte_carlo_ar1 import (generate_staggered_law_ar1_data)
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
N = 50
T = 20
rho = 0.2
num_individuals = 20

data = generate_staggered_law_ar1_data(N,T,rho,num_individuals)

In [3]:
data.columns

Index(['state', 'individual', 'time', 'value', 'state_2', 'state_3', 'state_4',
       'state_5', 'state_6', 'state_7', 'state_8', 'state_9', 'state_10',
       'state_11', 'state_12', 'state_13', 'state_14', 'state_15', 'state_16',
       'state_17', 'state_18', 'state_19', 'state_20', 'state_21', 'state_22',
       'state_23', 'state_24', 'state_25', 'state_26', 'state_27', 'state_28',
       'state_29', 'state_30', 'state_31', 'state_32', 'state_33', 'state_34',
       'state_35', 'state_36', 'state_37', 'state_38', 'state_39', 'state_40',
       'state_41', 'state_42', 'state_43', 'state_44', 'state_45', 'state_46',
       'state_47', 'state_48', 'state_49', 'state_50', 'time_1', 'time_2',
       'time_3', 'time_4', 'time_5', 'time_6', 'time_7', 'time_8', 'time_9',
       'time_10', 'time_11', 'time_12', 'time_13', 'time_14', 'time_15',
       'time_16', 'time_17', 'time_18', 'time_19', 'TREATMENT'],
      dtype='object')

In [10]:
!pip install linearmodels


  Obtaining dependency information for linearmodels from https://files.pythonhosted.org/packages/c8/19/be62804f9e7ffc1ca5b75ef1459ffe5834fb39cd39c95642ce70bc1702fb/linearmodels-5.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for Cython>=0.29.34 from https://files.pythonhosted.org/packages/60/d0/30f3ecd4dcafa87cd024a2687a6be7aa14f94d75c040192f8f8629ef8ad8/Cython-3.0.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyhdfe>=0.1 from https://files.pythonhosted.org/packages/2f/51/cb006fbc08c32f161035fb19ca718250eb5f6d0692ea6dcc1e62c3e556a2/pyhdfe-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for formulaic>=0.6.5 from https://files.pythonhosted.org/packages/ae/f0/ff275c0009f0037c381c6e3a94049b9792df7d7f437c5ccd0c7aefca5f22/formulaic-0.6.6-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB 1.3 MB/s eta 0:00:01
     

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.


In [11]:
from linearmodels.iv import IVGMM

In [15]:
def joint_gmm_regression(data):

    # Equation 1
    equation_1_formula = "value ~ state_2 + state_3 + state_4 + state_5 + state_6 + state_7 + state_8 + state_9 + state_10 + state_11 + state_12 + state_13 + state_14 + state_15 + state_16 + state_17 + state_18 + state_19 + state_20 + state_21 + state_22 + state_23 + state_24 + state_25 + state_26 + state_27 + state_28 + state_29 + state_30 + state_31 + state_32 + state_33 + state_34 + state_35 + state_36 + state_37 + state_38 + state_39 + state_40 + state_41 + state_42 + state_43 + state_44 + state_45 + state_46 + state_47 + state_48 + state_49 + state_50 + time_1 + time_2 + time_3 + time_4 + time_5 + time_6 + time_7 + time_8 + time_9 + time_10 + time_11 + time_12 + time_13 + time_14 + time_15 + time_16 + time_17 + time_18 + time_19 + 1"

    # Equation 2
    equation_2_formula = 'value ~ state_2 + state_3 + state_4 + state_5 + state_6 + state_7 + state_8 + state_9 + state_10 + state_11 + state_12 + state_13 + state_14 + state_15 + state_16 + state_17 + state_18 + state_19 + state_20 + state_21 + state_22 + state_23 + state_24 + state_25 + state_26 + state_27 + state_28 + state_29 + state_30 + state_31 + state_32 + state_33 + state_34 + state_35 + state_36 + state_37 + state_38 + state_39 + state_40 + state_41 + state_42 + state_43 + state_44 + state_45 + state_46 + state_47 + state_48 + state_49 + state_50 + time_1 + time_2 + time_3 + time_4 + time_5 + time_6 + time_7 + time_8 + time_9 + time_10 + time_11 + time_12 + time_13 + time_14 + time_15 + time_16 + time_17 + time_18 + time_19 + TREATMENT + 1'

    def moment_conditions_equation_1(params, data):
        # Extracting beta_state and beta_time from the params
        beta_state = params[:50]  # Assuming the first 50 parameters are for state dummies
        beta_time = params[50:]  # The remaining parameters are for time dummies

        # Extracting state and time dummy variables from the data
        state_dummies = data.iloc[:, 2:52].values  # Assuming columns 2 to 51 are state dummy variables
        time_dummies = data.iloc[:, 52:72].values  # Assuming columns 52 to 71 are time dummy variables

        # Calculating residuals for Equation 1
        residuals_equation_1 = data['value'] - np.dot(state_dummies, beta_state) - np.dot(time_dummies, beta_time)

        return residuals_equation_1
    
    def moment_conditions_equation_2(params, data):
        # Extracting beta_state, beta_time, and beta_treatment from the params
        beta_state = params[:50]  # Assuming the first 50 parameters are for state dummies
        beta_time = params[50:70]  # Assuming the next 20 parameters are for time dummies
        beta_treatment = params[-1]  # The last parameter is for TREATMENT (including intercept if present)

        # Extracting state, time, and treatment dummy variables from the data
        state_dummies = data.iloc[:, 2:52].values  # Assuming columns 2 to 51 are state dummy variables
        time_dummies = data.iloc[:, 52:72].values  # Assuming columns 52 to 71 are time dummy variables
        treatment_dummy = data['TREATMENT'].values

    # Calculating residuals for Equation 2
        residuals_equation_2 = data['value'] - np.dot(state_dummies, beta_state) - np.dot(time_dummies, beta_time) - beta_treatment * treatment_dummy

        return residuals_equation_2
    
        # Initial guess for beta
    initial_beta = np.zeros(71)  # Assuming 50 state dummies, 20 time dummies, and 1 for TREATMENT

    # Stack the moment conditions
    moments_equations = [moment_conditions_equation_1, moment_conditions_equation_2]

    # Estimate beta using Joint GMM
    gmm_model_joint = IVGMM(initial_beta, moments_equations,  args=(data,))
    result_joint_gmm = gmm_model_joint.fit()

    return result_joint_gmm


N = 50
T = 20
rho = 0.2
num_individuals = 20
data = generate_staggered_law_ar1_data(N,T,rho,num_individuals)

result_joint_gmm = joint_gmm_regression(data)
print(result_joint_gmm.summary())


TypeError: IVGMM.__init__() missing 2 required positional arguments: 'endog' and 'instruments'

In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM

# Assuming df is your DataFrame with 'Y', 'group', 'period', 'D'
# 'instrument1' is a list of lists, each sublist is a different instrument for the first equation
# 'instrument2' is a list for the second equation

N = 50
T = 20
rho = 0.2
num_individuals = 20

df = generate_staggered_law_ar1_data(N,T,rho,num_individuals)

class TwoStageGMM(GMM):
    def __init__(self, endog, exog, instrument1, instrument2):
        # Flatten instrument1 list of lists
        instrument1_flat = np.column_stack(instrument1)
        # Prepare the combined instrument matrix
        instruments = np.column_stack((instrument1_flat, instrument2))
        super(TwoStageGMM, self).__init__(endog, exog, instruments=instruments)
        self.instrument1_flat = instrument1_flat
        self.instrument2 = instrument2

    def momcond(self, params):
        # Unpack parameters for both equations
        lambda_g, gamma_p, treatment_effect = params[:len(params)//2]
        lambda_g2, gamma_p2, treatment_effect2 = params[len(params)//2:]

        # First equation moment conditions
        group_dummies = pd.get_dummies(self.exog['state'], drop_first=True)
        time_dummies = pd.get_dummies(self.exog['time'], drop_first=True)
        err1 = self.endog - (group_dummies @ lambda_g + time_dummies @ gamma_p + self.exog['TREATMENT'] * treatment_effect)
        moment1 = err1[:, None] * self.instrument1_flat

        # Second equation moment conditions
        err2 = self.endog - (group_dummies @ lambda_g2 + time_dummies @ gamma_p2 + self.exog['TREATMENT'] * treatment_effect2)
        moment2 = err2[:, None] * self.instrument2[:, None]

        return np.column_stack((moment1, moment2))

# Prepare the data
endog = df['value']
exog = df[['state_2', 'state_3', 'state_4',
       'state_5', 'state_6', 'state_7', 'state_8', 'state_9', 'state_10',
       'state_11', 'state_12', 'state_13', 'state_14', 'state_15', 'state_16',
       'state_17', 'state_18', 'state_19', 'state_20', 'state_21', 'state_22',
       'state_23', 'state_24', 'state_25', 'state_26', 'state_27', 'state_28',
       'state_29', 'state_30', 'state_31', 'state_32', 'state_33', 'state_34',
       'state_35', 'state_36', 'state_37', 'state_38', 'state_39', 'state_40',
       'state_41', 'state_42', 'state_43', 'state_44', 'state_45', 'state_46',
       'state_47', 'state_48', 'state_49', 'state_50', 'time_1', 'time_2',
       'time_3', 'time_4', 'time_5', 'time_6', 'time_7', 'time_8', 'time_9',
       'time_10', 'time_11', 'time_12', 'time_13', 'time_14', 'time_15',
       'time_16', 'time_17', 'time_18', 'time_19', 'TREATMENT']]
instrument1 = df[['state_2', 'state_3', 'state_4',
       'state_5', 'state_6', 'state_7', 'state_8', 'state_9', 'state_10',
       'state_11', 'state_12', 'state_13', 'state_14', 'state_15', 'state_16',
       'state_17', 'state_18', 'state_19', 'state_20', 'state_21', 'state_22',
       'state_23', 'state_24', 'state_25', 'state_26', 'state_27', 'state_28',
       'state_29', 'state_30', 'state_31', 'state_32', 'state_33', 'state_34',
       'state_35', 'state_36', 'state_37', 'state_38', 'state_39', 'state_40',
       'state_41', 'state_42', 'state_43', 'state_44', 'state_45', 'state_46',
       'state_47', 'state_48', 'state_49', 'state_50', 'time_1', 'time_2',
       'time_3', 'time_4', 'time_5', 'time_6', 'time_7', 'time_8', 'time_9',
       'time_10', 'time_11', 'time_12', 'time_13', 'time_14', 'time_15',
       'time_16', 'time_17', 'time_18', 'time_19']]
instrument2 = df[["TREATMENT"]]
# Initialize and fit the model
initial_params = np.array([0.1] * (len(df['state'].unique()) + len(df['time'].unique()) + 1) * 2)  # Adjust as necessary
model = TwoStageGMM(endog, exog, instrument1, instrument2)
result = model.fit(start_params=initial_params)

# Display results
print(result.summary())


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 20000